© 2024 Brian Butka. All rights reserved.

In [8]:
def check_binary_validity(binary_num):
    # Check if the input is a valid binary number with length between 4 and 11 bits
    return len(binary_num) in range(4, 12) and all(bit in '01' for bit in binary_num)

def store_binary_bits(binary_num):
    # Initialize a list with 11 locations, each set to 2
    bit_storage = [2] * 11
    # Store the binary bits in the variable bit slots, LSB on the right
    for i, bit in enumerate(reversed(binary_num)):
        bit_storage[i] = int(bit)
    return bit_storage

def calculate_parity_bits(transmitted_with_zeros, bit_length):
    # Calculate parity bits based on given dependencies
    parity_positions = [0, 1, 3]  # Parity bit positions are 1, 2, and 4 (zero-indexed)
    if bit_length > 4:
        parity_positions.append(7)  # Parity bit position for P8

    dependencies = {
        0: [i for i in range(15) if (i + 1) & 1 and i != 0],
        1: [i for i in range(15) if (i + 1) & 2 and i != 1],
        3: [i for i in range(15) if (i + 1) & 4 and i != 3],
        7: [i for i in range(15) if (i + 1) & 8 and i != 7]
    }

    parity_bits = {}
    for p in parity_positions:
        if bit_length <= 4 and p >= 7:
            continue
        parity_sum = sum(transmitted_with_zeros[i] for i in dependencies[p] if i < bit_length + len(parity_positions) and i < 15)
        parity_bits[p] = parity_sum % 2
        used_bits = [i for i in dependencies[p] if i < bit_length + len(parity_positions) and i < 15]
        print_parity_info(p + 1, [i + 1 for i in used_bits], transmitted_with_zeros, parity_bits[p])

    return parity_bits

def print_parity_info(parity_pos, used_bits, transmitted_with_zeros, parity_value):
    used_bits_str = ' '.join(f"{i:2}" for i in used_bits)
    values_used_str = ' '.join(f"{transmitted_with_zeros[i-1]:2}" for i in used_bits)
    print(f"\nParity bit P{parity_pos} calculation:")
    print(f"Bit positions used: {used_bits_str}")
    print(f"Values used:        {values_used_str}")
    print(f"P{parity_pos} = {parity_value}")

def display_bits(binary_num, bit_storage):
    bit_length = len(binary_num)
    bit_positions = ' '.join(f"{i:2}" for i in range(bit_length, 0, -1))
    data_bits = ' '.join(f"{bit_storage[i-1]:2}" for i in range(bit_length, 0, -1))
    print(f"\nBit position: {bit_positions}")
    print(f"Data bits:    {data_bits}")

def create_transmitted_with_P(bit_storage):
    transmitted = ["P"] * 15
    data_index = 0
    for i in range(15):
        if i not in [0, 1, 3, 7]:
            if data_index < len(bit_storage):
                transmitted[i] = bit_storage[data_index]
                data_index += 1
    return transmitted

def display_transmitted_with_P(transmitted, bit_length):
    max_bit = 7 if bit_length == 4 else 4 + bit_length
    bit_positions = ' '.join(f"{i:2}" for i in range(max_bit, 0, -1))
    data_bits = ' '.join(f"{transmitted[i-1]:2}" if transmitted[i-1] != "P" else " P" for i in range(max_bit, 0, -1))
    print(f"\nTransmitted bit positions: {bit_positions}")
    print(f"Transmitted data bits:     {data_bits}")
    transmitted_with_zeros = [0 if bit == "P" else bit for bit in transmitted]
    return max_bit, transmitted_with_zeros

def create_transmitted(transmitted_with_zeros, parity_bits):
    transmitted = transmitted_with_zeros[:]
    for pos, value in parity_bits.items():
        transmitted[pos] = value
    return transmitted

def display_transmitted(transmitted, bit_length):
    max_bit = 7 if bit_length == 4 else 4 + bit_length
    bit_positions = ' '.join(f"{i:2}" for i in range(max_bit, 0, -1))
    data_bits = ' '.join(f"{transmitted[i-1]:2}" for i in range(max_bit, 0, -1))
    print(f"\nTransmitted bit positions: {bit_positions}")
    print(f"Transmitted data bits:     {data_bits}")
    return max_bit

def flip_bit(transmitted, bit_pos):
    transmitted[bit_pos - 1] ^= 1  # Flip the specified bit
    return transmitted

def check_parity(transmitted, max_bits):
    parity_positions = [0, 1, 3] if max_bits <= 7 else [0, 1, 3, 7]
    dependencies = {
        0: [i for i in range(15) if (i + 1) & 1],
        1: [i for i in range(15) if (i + 1) & 2],
        3: [i for i in range(15) if (i + 1) & 4],
        7: [i for i in range(15) if (i + 1) & 8]
    }
    error_position = 0
    for p in parity_positions:
        if max_bits <= 7 and p >= 7:
            continue
        parity_sum = sum(transmitted[i] for i in dependencies[p] if i < max_bits)
        parity_value = parity_sum % 2
        error_position += p + 1 if parity_value != 0 else 0
        used_bits_str = ' '.join(f"{i+1:2}" for i in dependencies[p] if i < max_bits)
        values_used_str = ' '.join(f"{transmitted[i]:2}" for i in dependencies[p] if i < max_bits)
        print(f"\nCheck parity bit P{p + 1} calculation:")
        print(f"Bit positions used: {used_bits_str}")
        print(f"Values used:        {values_used_str}")
        print(f"P{p + 1} = {parity_value}")
    return error_position

def extract_data_bits(transmitted, bit_length):
    data_positions = [i for i in range(15) if i not in [0, 1, 3, 7]]
    data_bits = [transmitted[data_positions[i]] for i in range(bit_length)]
    return list(reversed(data_bits))

def display_corrected_data(data_bits):
    bit_length = len(data_bits)
    bit_positions = ' '.join(f"{i:2}" for i in range(bit_length, 0, -1))
    data_bits_str = ' '.join(f"{bit:2}" for bit in data_bits)
    print(f"\nCorrected data bit positions: {bit_positions}")
    print(f"Corrected data bits:          {data_bits_str}")

def main():
    binary_num = input("Enter a binary number (4 to 11 bits): ")
    if not check_binary_validity(binary_num):
        print("Invalid binary number. Please enter a binary number with 4 to 11 bits.")
        return
    bit_storage = store_binary_bits(binary_num)
    bit_length = len(binary_num)
    display_bits(binary_num, bit_storage)
    transmitted_with_P = create_transmitted_with_P(bit_storage)
    max_bit, transmitted_with_zeros = display_transmitted_with_P(transmitted_with_P, bit_length)
    parity_bits = calculate_parity_bits(transmitted_with_zeros, bit_length)
    transmitted = create_transmitted(transmitted_with_zeros, parity_bits)
    display_transmitted(transmitted, bit_length)
    error_bit = int(input(f"Enter the bit position (1 to {max_bit}) to flip (or 0 for no error): "))
    if error_bit != 0:
        transmitted = flip_bit(transmitted, error_bit)
        print("\nReceived data after flipping the bit:")
        display_transmitted(transmitted, bit_length)
    error_position = check_parity(transmitted, max_bit)
    if error_position == 0:
        print("\nNo Error")
    else:
        print(f"\nError detected at bit position (P8 P4 P2 P1): {error_position}")
        transmitted = flip_bit(transmitted, error_position)
        print("\nCorrected received bits:")
        display_transmitted(transmitted, bit_length)
    corrected_data_bits = extract_data_bits(transmitted, bit_length)
    display_corrected_data(corrected_data_bits)

if __name__ == "__main__":
    main()


Enter a binary number (4 to 11 bits): 11100001111

Bit position: 11 10  9  8  7  6  5  4  3  2  1
Data bits:     1  1  1  0  0  0  0  1  1  1  1

Transmitted bit positions: 15 14 13 12 11 10  9  8  7  6  5  4  3  2  1
Transmitted data bits:      1  1  1  0  0  0  0  P  1  1  1  P  1  P  P

Parity bit P1 calculation:
Bit positions used:  3  5  7  9 11 13 15
Values used:         1  1  1  0  0  1  1
P1 = 1

Parity bit P2 calculation:
Bit positions used:  3  6  7 10 11 14 15
Values used:         1  1  1  0  0  1  1
P2 = 1

Parity bit P4 calculation:
Bit positions used:  5  6  7 12 13 14 15
Values used:         1  1  1  0  1  1  1
P4 = 0

Parity bit P8 calculation:
Bit positions used:  9 10 11 12 13 14 15
Values used:         0  0  0  0  1  1  1
P8 = 1

Transmitted bit positions: 15 14 13 12 11 10  9  8  7  6  5  4  3  2  1
Transmitted data bits:      1  1  1  0  0  0  0  1  1  1  1  0  1  1  1
Enter the bit position (1 to 15) to flip (or 0 for no error): 14

Received data after flipping th

In [ ]:

    %%javascript
    function hideCode(){
        var code_cells = document.querySelectorAll('div.input');
        for (var i = 0; i < code_cells.length; i++) {
            code_cells[i].style.display = 'none';
        }
    }
    hideCode();
    